# Generate point-in-time best execution data using dedicated service - F#

### Overview
Best execution aims to measure the performance of executed trades, an execution algorithm or an execution venue.

This sample is based on an `point-in-time` approach and designed to highlight individual trades within market activity over a look back period. 

<div class="alert alert-block alert-info">
    <b>Note:</b> To explore the interval approach, suitable to analyze market activity over a look back period. Please refer to <i>bestex-interval</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** best execution *point-in-time* results.

### Inputs/outputs
Best execution PIT sample requires instrument's identifier, **a list of trades** and time window as per inputs. Results are as follows:
* a snapshot of occured trades around the given trade based on the input time window
* plot the request result and export data

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Packages required
1. Systemathics:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source:
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***


# Run BestEx Point-in-time sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.9.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: FSharp.Data"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Tick.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
// Plot charts
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data

#### 3.1 Input trades import
We assume our trades are in a *csv file* that we imported in the same folder as the current data sample.

We aim to to ensure that it was the best possible execution for each and every trade. To do so, we have to retrieve the *trades as a market snapshot* and mark out the *individual trades*.

The following code snippets import *individual trades* from the *csv file*:

In [ ]:
// Load the trades from the csv input file
open FSharp.Data

type InputTrades = CsvProvider<"input_trades.csv">
let inputTrades = 
    (new InputTrades ()).Rows
    |> Seq.map (fun i -> { Timestamp = i.Timestamp.DateTime; Price = double i.Price; Size = int64 i.Size; Flag = i.Flag })
    |> Seq.sortBy (fun i -> i.Timestamp)
    |> Seq.toArray


// Display the loaded trades
inputTrades
|> display


### Step 3: Create and process request
To request *tick trades* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick trades request parameters

#### 3.1 Instrument selection

In [ ]:
// Set the instrument  identifier: a ticker and an exchange code
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Best execution interval selection

In [ ]:
// Set the time window to delimit the input trade in seconds
let window = TimeSpan.FromSeconds(5. * 60.)

#### 3.3 Time period delimitation

The following code snippets delimits the best execution time interval around each input trade based on the previously chosen time window.

In [ ]:
// Get the first and the last dates from the input trades


let firstDate = (inputTrades |> Array.head).Timestamp.Date
let lastDate = (inputTrades |> Array.last).Timestamp.Date

// Build the tick trades date interval (we are using Google date time format)
let dateIntervals = 
    new DateInterval(
        StartDate = new Date ( Year = firstDate.Year, Month = firstDate.Month, Day = firstDate.Day ),
        EndDate = new Date ( Year = lastDate.Year, Month = lastDate.Month, Day = lastDate.Day )
    )

// Get the first and the last timestamps from the input trades
let firstTime = (inputTrades |> Array.minBy (fun t -> t.Timestamp.TimeOfDay)).Timestamp.TimeOfDay.Add(-window)
let lastTime = (inputTrades |> Array.maxBy( fun t -> t.Timestamp.TimeOfDay)).Timestamp.TimeOfDay.Add(window)

// Build the tick trades time interval (we are using Google date time format)
let timeInterval = 
    new TimeInterval( 
        StartTime = new TimeOfDay ( Hours = firstTime.Hours, Minutes = firstTime.Minutes, Seconds = firstTime.Seconds ),
        EndTime = new TimeOfDay ( Hours = lastTime.Hours, Minutes = lastTime.Minutes, Seconds = lastTime.Seconds )
    )

#### 3.4 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Generate constraints based on the previous time selection
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

In [ ]:
// Generate the tick trades request
let request = new TickTradesRequest ( Constraints = constraints )
let identifier = new Identifier ( Exchange = exchange, Ticker = ticker)
request.Identifiers.Add([| identifier |]) 

In [ ]:
// Instantiate the tick trades service
let service = new TickTradesService.TickTradesServiceClient(channel)

### Step 4: Visualize data

#### 4.1 Retrieve occured trades

In [ ]:
// Get the trades
open FSharp.Control.Tasks.V2
open System.Collections.Generic

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()

In [ ]:
// Get the trades
let trades = new List<Trade>()

let call = service.TickTrades(request, headers)
let e = call.ResponseStream.ReadAllAsync()

let add (collection : List<Trade>) (item : TickTradesResponse) =
    if( item |> isNull |> not && item.Trade |> isNull |> not ) then
        collection.Add item.Trade
    
iterAsync e (add trades)


In [ ]:
// Display trades
//trades |> display
let split (t: List<Trade>) = 
    t
    |> Seq.toList
    |> List.map (fun t -> (t.TimeStamp.ToDateTime(), t.Price))
    |> List.unzip

let split' (t: InputTrade []) = 
    t
    |> Array.map (fun t -> (t.Timestamp, t.Price))
    |> Array.unzip

let (mktKeys, mktValues) = split trades
let (inputKeys, inputValues) = split' inputTrades


#### 4.3 Plot individual trades within market snapshot
Display trade prices and highlight the individual trades over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.
Plot a simple line graph with the trade prices and mark out the individual trades as follows: 

In [ ]:
// Create the graph
[
    new Scattergl(
        name = "Market trades", 
        x = mktKeys, 
        y = mktValues
        )
    new Scattergl(
        name = "Input trades", 
        x = inputKeys, 
        y = inputValues, 
        mode="markers"
        )
]
|> Chart.Plot
|> Chart.WithTitle $"Market snapshot and input trades for {ticker}"
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "Price"
|> Chart.WithWidth  1500
|> Chart.WithHeight  800
|> display